<a href="https://colab.research.google.com/github/LakshmanYES/My-School-Projects/blob/main/Sai_Lakshman_Ethakatla_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Spark init
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/default-java"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("My Spark App").setMaster("local")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [ ]:
from pyspark.sql.types import *

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
!wget --continue http://saifmohammad.com/WebDocs/Jan9-2012-tweets-clean.txt.zip  -O /content/drive/MyDrive/data/Jan9-2012-tweets-clean.txt.zip

--2022-04-22 02:41:31--  http://saifmohammad.com/WebDocs/Jan9-2012-tweets-clean.txt.zip
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1128895 (1.1M) [application/zip]
Saving to: ‘/content/drive/MyDrive/data/Jan9-2012-tweets-clean.txt.zip’

/content/drive/MyDr 100%[===================>]   1.08M  4.90MB/s    in 0.2s    

2022-04-22 02:41:32 (4.90 MB/s) - ‘/content/drive/MyDrive/data/Jan9-2012-tweets-clean.txt.zip’ saved [1128895/1128895]



In [ ]:
!unzip /content/drive/MyDrive/data/Jan9-2012-tweets-clean.txt.zip -d /content/drive/MyDrive/data

Archive:  /content/drive/MyDrive/data/Jan9-2012-tweets-clean.txt.zip
  inflating: /content/drive/MyDrive/data/Jan9-2012-tweets-clean.txt  
   creating: /content/drive/MyDrive/data/__MACOSX/
  inflating: /content/drive/MyDrive/data/__MACOSX/._Jan9-2012-tweets-clean.txt  


In [ ]:
tweets_schema = StructType([
        StructField("content", StringType(), True),
        StructField("emotion", StringType(), True)
    ])

In [ ]:
df_tweet_clean = spark.read.csv('/content/drive/MyDrive/data/Jan9-2012-tweets-clean.txt', sep=':: ',
                         schema = tweets_schema, header=False)

In [ ]:
df_tweet_clean.show()

+--------------------+--------+
|             content| emotion|
+--------------------+--------+
|14535304881701273...|surprise|
|14427963802425753...| sadness|
|14049958528511180...|     joy|
|14520757827050700...|     joy|
|13950214639047065...| disgust|
|14604269689988710...|     joy|
|14549256960908492...|     joy|
|14590395522915123...|     joy|
|14271761323406950...|    fear|
|14418382287392768...|   anger|
|14595999580305817...| sadness|
|14291809701003673...|surprise|
|14218640322317926...|surprise|
|14570353827684352...| sadness|
|14594775270936985...|     joy|
|13895892187546419...|   anger|
|14616042380736102...|     joy|
|13911997107365888...|surprise|
|13992503437754777...|     joy|
|14596543439765913...| sadness|
+--------------------+--------+
only showing top 20 rows



In [ ]:
df_part_A = df_tweet_clean.select('content','emotion')

In [ ]:
df_part_A.show()

+--------------------+--------+
|             content| emotion|
+--------------------+--------+
|14535304881701273...|surprise|
|14427963802425753...| sadness|
|14049958528511180...|     joy|
|14520757827050700...|     joy|
|13950214639047065...| disgust|
|14604269689988710...|     joy|
|14549256960908492...|     joy|
|14590395522915123...|     joy|
|14271761323406950...|    fear|
|14418382287392768...|   anger|
|14595999580305817...| sadness|
|14291809701003673...|surprise|
|14218640322317926...|surprise|
|14570353827684352...| sadness|
|14594775270936985...|     joy|
|13895892187546419...|   anger|
|14616042380736102...|     joy|
|13911997107365888...|surprise|
|13992503437754777...|     joy|
|14596543439765913...| sadness|
+--------------------+--------+
only showing top 20 rows



In [ ]:

emotion_sample=["joy","anger","surprise"]


df_part_B=df_tweet_clean.filter(df_tweet_clean.emotion.isin(emotion_sample))

In [ ]:
df_part_B.show()

+--------------------+--------+
|             content| emotion|
+--------------------+--------+
|14535304881701273...|surprise|
|14049958528511180...|     joy|
|14520757827050700...|     joy|
|14604269689988710...|     joy|
|14549256960908492...|     joy|
|14590395522915123...|     joy|
|14418382287392768...|   anger|
|14291809701003673...|surprise|
|14218640322317926...|surprise|
|14594775270936985...|     joy|
|13895892187546419...|   anger|
|14616042380736102...|     joy|
|13911997107365888...|surprise|
|13992503437754777...|     joy|
|14588083752573337...|     joy|
|14516003874551398...|     joy|
|14050796153249382...|     joy|
|14273902091975065...|     joy|
|14578161902485913...|     joy|
|14427886515245875...|     joy|
+--------------------+--------+
only showing top 20 rows



In [ ]:
from pyspark.ml.feature import HashingTF, Tokenizer
import re
from pyspark.sql.functions import udf, length
from pyspark.sql.types import StringType, ArrayType
from pyspark.ml.feature import StopWordsRemover
filter = re.compile(r"^[a-zA-Z]+$")
match_udf = udf(lambda tokens: [token for token in tokens if filter.match(token)], ArrayType(StringType()))
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))

In [ ]:
def to_features(df):
  tokenizer = Tokenizer(inputCol="content", outputCol="words")
  wordsData = tokenizer.transform(df)
  df_matched = wordsData.withColumn("words_matched", match_udf("words")). \
    select('emotion', 'words_matched')
  # Remove stop words
  remover = StopWordsRemover(inputCol='words_matched', outputCol='words_clean')
  df_words_clean = remover.transform(df_matched). \
    select('emotion', 'words_clean')
  # Stem text
  df_stemmed = df_words_clean.withColumn("words_stemmed", stemmer_udf("words_clean")). \
    select('emotion', 'words_stemmed')

  # Generate vectors
  hashingTF = HashingTF(inputCol="words_stemmed", outputCol="rawFeatures") 
  featurizedData = hashingTF.transform(df_stemmed)
  return featurizedData

In [ ]:
(training_part_A, test_part_A) = df_part_A.randomSplit([0.8, 0.2], seed = 100)
print (training_part_A.first())

Row(content='136060125965328385:\tI Hate People With Crusty Nasty Lips \t', emotion='disgust')


In [ ]:
print("The total number of records (count) of the training data",training_part_A.count())

The total number of records (count) of the training data 16836


In [ ]:
df_train_features_part_A = to_features(training_part_A)
df_train_features_part_A.show()

+-------+--------------------+--------------------+
|emotion|       words_stemmed|         rawFeatures|
+-------+--------------------+--------------------+
|disgust|[hate, peopl, cru...|(262144,[71195,72...|
|disgust|[cat, parti, next...|(262144,[35315,45...|
|disgust|   [laugh, everyday]|(262144,[97914,25...|
|disgust|[dumb, fuck, two,...|(262144,[21823,24...|
|disgust|[see, coupl, pda,...|(262144,[8538,768...|
|disgust|[sister, black, kid]|(262144,[5451,752...|
|  anger|[ridicul, mad, me...|(262144,[32055,61...|
|disgust|[sack, similar, c...|(262144,[29526,15...|
|disgust|[yall, dont, unde...|(262144,[31015,76...|
|disgust|[mlk, fought, bla...|(262144,[31536,14...|
|disgust|[hoe, tri, hard, ...|(262144,[2437,777...|
|disgust|[white, girl, tal...|(262144,[13781,75...|
|disgust|[watch, guy, scra...|(262144,[46846,16...|
|  anger|[meant, rt, sarah...|(262144,[59225,77...|
|disgust|[peopl, live, cho...|(262144,[20488,33...|
|disgust|[ugh, bum, nigga,...|(262144,[89717,23...|
|disgust|[la

In [ ]:
df_Allemotions_in_file = test_part_A.select("emotion").distinct()
df_Allemotions_in_file.show()

+--------+
| emotion|
+--------+
|     joy|
|   anger|
|    fear|
|surprise|
| sadness|
| disgust|
+--------+



In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol = 'emotion', outputCol = 'emotion_index')

In [ ]:
df_Allemotions_indexed = indexer.fit(df_Allemotions_in_file).transform(df_Allemotions_in_file)
df_Allemotions_indexed.show()

df_train_data_features_part_A_indexed = df_train_features_part_A.join(df_Allemotions_indexed, 'emotion')
df_train_data_features_part_A_indexed.show()

+--------+-------------+
| emotion|emotion_index|
+--------+-------------+
|     joy|          3.0|
|   anger|          0.0|
|    fear|          2.0|
|surprise|          5.0|
| sadness|          4.0|
| disgust|          1.0|
+--------+-------------+

+-------+--------------------+--------------------+-------------+
|emotion|       words_stemmed|         rawFeatures|emotion_index|
+-------+--------------------+--------------------+-------------+
|disgust|[hate, peopl, cru...|(262144,[71195,72...|          1.0|
|disgust|[cat, parti, next...|(262144,[35315,45...|          1.0|
|disgust|   [laugh, everyday]|(262144,[97914,25...|          1.0|
|disgust|[dumb, fuck, two,...|(262144,[21823,24...|          1.0|
|disgust|[see, coupl, pda,...|(262144,[8538,768...|          1.0|
|disgust|[sister, black, kid]|(262144,[5451,752...|          1.0|
|  anger|[ridicul, mad, me...|(262144,[32055,61...|          0.0|
|disgust|[sack, similar, c...|(262144,[29526,15...|          1.0|
|disgust|[yall, dont, u

In [ ]:
# Naive Bayes classifier
from pyspark.ml.classification import NaiveBayes
   
nb = NaiveBayes(labelCol="emotion_index",\
    featuresCol="rawFeatures", smoothing=1.0,\
    modelType="multinomial")
model = nb.fit(df_train_data_features_part_A_indexed)

In [ ]:
df_test_data_features_part_A = to_features(test_part_A)
df_test_data_features_part_A.show()

+-------+--------------------+--------------------+
|emotion|       words_stemmed|         rawFeatures|
+-------+--------------------+--------------------+
|disgust|[decid, get, coff...|(262144,[22962,31...|
|disgust|[realli, forgot, ...|(262144,[23205,60...|
|  anger|[hear, music, mak...|(262144,[25923,89...|
|disgust|[honesti, asham, ...|(262144,[33053,51...|
|disgust|[ppl, eat, octopu...|(262144,[69217,20...|
|disgust|       [fuck, wrong]|(262144,[201440,2...|
|  anger|[ughhhhhhhhhh, d,...|(262144,[5451,106...|
|  anger|[sentenc, transla...|(262144,[2896,971...|
|  anger|[resent, store, h...|(262144,[25885,40...|
|  anger|[righteous, never...|(262144,[3571,113...|
|  anger|[god, teach, us, ...|(262144,[39498,60...|
|disgust|[abus, christi, w...|(262144,[13188,30...|
|  anger|[omg, leav, alon,...|(262144,[4321,700...|
|disgust|[awckward, moment...|(262144,[114972,1...|
|  anger|  [tri, hard, break]|(262144,[2437,124...|
|  anger|[hate, peopl, lik...|(262144,[31571,72...|
|  anger|[as

In [ ]:
df_test_data_features_part_A_indexed = df_test_data_features_part_A.join(df_Allemotions_indexed, 'emotion')
df_test_data_features_part_A_indexed.show()

+-------+--------------------+--------------------+-------------+
|emotion|       words_stemmed|         rawFeatures|emotion_index|
+-------+--------------------+--------------------+-------------+
|    joy|[oh, seem, weeken...|(262144,[18184,13...|          3.0|
|    joy|[joy, around, us,...|(262144,[64859,10...|          3.0|
|    joy|[figur, someth, c...|(262144,[31413,12...|          3.0|
|    joy|[done, one, conti...|(262144,[21823,25...|          3.0|
|    joy|[one, common, par...|(262144,[21823,85...|          3.0|
|    joy|[love, dramat, sa...|(262144,[97564,15...|          3.0|
|    joy|[give, joy, spend...|(262144,[76764,10...|          3.0|
|    joy|[leav, hous, driv...|(262144,[59776,98...|          3.0|
|    joy|[shower, back, co...|(262144,[132270,1...|          3.0|
|    joy|[famili, live, sc...|(262144,[4790,226...|          3.0|
|    joy|[know, dissert, t...|(262144,[16673,13...|          3.0|
|    joy|[anoth, day, amer...|(262144,[113722,1...|          3.0|
|    joy|[

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictions = model.transform(df_test_data_features_part_A_indexed)
predictions.select("emotion", "emotion_index", 
    "probability", "prediction").show()
evaluator =\
    MulticlassClassificationEvaluator(labelCol="emotion_index",\
    predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+-------+-------------+--------------------+----------+
|emotion|emotion_index|         probability|prediction|
+-------+-------------+--------------------+----------+
|    joy|          3.0|[4.87901133922655...|       3.0|
|    joy|          3.0|[4.58809973435617...|       3.0|
|    joy|          3.0|[0.01730116890444...|       3.0|
|    joy|          3.0|[2.83085343041881...|       3.0|
|    joy|          3.0|[0.00108184998744...|       3.0|
|    joy|          3.0|[8.62202165197623...|       3.0|
|    joy|          3.0|[1.67025392169679...|       3.0|
|    joy|          3.0|[1.05466560469644...|       3.0|
|    joy|          3.0|[6.26886719002542...|       3.0|
|    joy|          3.0|[2.66352103997177...|       3.0|
|    joy|          3.0|[1.44740020609555...|       3.0|
|    joy|          3.0|[7.70003215673989...|       3.0|
|    joy|          3.0|[2.13968550064955...|       3.0|
|    joy|          3.0|[5.0100372077993E...|       3.0|
|    joy|          3.0|[0.00108129665967...|    

In [ ]:
(training_part_B, test_part_B) = df_part_B.randomSplit([0.8, 0.2], seed = 50)
print (training_part_B.first())

Row(content="136211499625627648:\tI was ridiculous mad with @Jilian_Lovex  and @BritUntitled . Seriously, don't mess with important school papers.\t", emotion='anger')


In [ ]:
print("The total number of records (count) of the training data",training_part_B.count())

The total number of records (count) of the training data 10894


In [ ]:
df_train_features_part_B = to_features(training_part_B)
df_train_features_part_B.show()

+-------+--------------------+--------------------+
|emotion|       words_stemmed|         rawFeatures|
+-------+--------------------+--------------------+
|  anger|[ridicul, mad, me...|(262144,[32055,61...|
|  anger|[hear, music, mak...|(262144,[25923,89...|
|  anger|[meant, rt, sarah...|(262144,[59225,77...|
|  anger|[automat, respons...|(262144,[69395,12...|
|  anger|[resent, abus, es...|(262144,[13188,40...|
|  anger|[automat, respons...|(262144,[69395,12...|
|  anger|[ughhhhhhhhhh, d,...|(262144,[5451,106...|
|  anger|[sentenc, transla...|(262144,[2896,971...|
|  anger|[lose, pen, lid, ...|(262144,[32055,91...|
|  anger|[righteous, never...|(262144,[3571,113...|
|  anger|[learn, righteous...|(262144,[3571,113...|
|  anger|[god, teach, us, ...|(262144,[39498,60...|
|  anger|[rt, righteous, n...|(262144,[3571,774...|
|  anger|[chang, think, wo...|(262144,[24031,84...|
|  anger|[buri, kill, buri...|(262144,[46561,57...|
|  anger|[bottom, hope, ha...|(262144,[37521,10...|
|  anger|[pr

In [ ]:
df_Allemotions_in_file = test_part_B.select("emotion").distinct()
df_Allemotions_in_file.show()

+--------+
| emotion|
+--------+
|     joy|
|   anger|
|surprise|
+--------+



In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol = 'emotion', outputCol = 'emotion_index')

In [ ]:
df_Allemotions_indexed = indexer.fit(df_Allemotions_in_file).transform(df_Allemotions_in_file)
df_Allemotions_indexed.show()

df_train_data_features_part_B_indexed = df_train_features_part_B.join(df_Allemotions_indexed, 'emotion')
df_train_data_features_part_B_indexed.show()

+--------+-------------+
| emotion|emotion_index|
+--------+-------------+
|     joy|          1.0|
|   anger|          0.0|
|surprise|          2.0|
+--------+-------------+

+-------+--------------------+--------------------+-------------+
|emotion|       words_stemmed|         rawFeatures|emotion_index|
+-------+--------------------+--------------------+-------------+
|  anger|[ridicul, mad, me...|(262144,[32055,61...|          0.0|
|  anger|[hear, music, mak...|(262144,[25923,89...|          0.0|
|  anger|[meant, rt, sarah...|(262144,[59225,77...|          0.0|
|  anger|[automat, respons...|(262144,[69395,12...|          0.0|
|  anger|[resent, abus, es...|(262144,[13188,40...|          0.0|
|  anger|[automat, respons...|(262144,[69395,12...|          0.0|
|  anger|[ughhhhhhhhhh, d,...|(262144,[5451,106...|          0.0|
|  anger|[sentenc, transla...|(262144,[2896,971...|          0.0|
|  anger|[lose, pen, lid, ...|(262144,[32055,91...|          0.0|
|  anger|[righteous, never...|(2

In [ ]:
# Naive Bayes classifier
from pyspark.ml.classification import NaiveBayes
   
nb = NaiveBayes(labelCol="emotion_index",\
    featuresCol="rawFeatures", smoothing=1.0,\
    modelType="multinomial")
model = nb.fit(df_train_data_features_part_B_indexed)

In [ ]:
df_test_data_features_part_B = to_features(test_part_B)
df_test_data_features_part_B.show()

+-------+--------------------+--------------------+
|emotion|       words_stemmed|         rawFeatures|
+-------+--------------------+--------------------+
|  anger|[resent, store, h...|(262144,[25885,40...|
|  anger|[buri, kill, buri...|(262144,[46561,57...|
|  anger|[assault, serv, u...|(262144,[70953,85...|
|  anger|[one, day, go, wr...|(262144,[21823,14...|
|  anger| [nano, recal, find]|(262144,[91878,10...|
|  anger|[wish, go, bitch,...|(262144,[2192,181...|
|  anger|[fact, matter, fr...|(262144,[1512,608...|
|  anger|[take, larger, pe...|(262144,[15769,33...|
|  anger|[get, insid, buil...|(262144,[33140,41...|
|  anger|[snipe, cod, come...|(262144,[13781,15...|
|  anger|[manag, sinc, peo...|(262144,[1218,928...|
|  anger|[fair, warn, b, l...|(262144,[21823,23...|
|  anger|[bud, one, put, b...|(262144,[5765,184...|
|  anger|          [oh, solv]|(262144,[18184,22...|
|  anger|[tri, get, stuff,...|(262144,[19818,32...|
|  anger|[skyrim, glitch, ...|(262144,[51471,89...|
|  anger|[ju

In [ ]:
df_test_data_features_part_B_indexed = df_test_data_features_part_B.join(df_Allemotions_indexed, 'emotion')
df_test_data_features_part_B_indexed.show()

+-------+--------------------+--------------------+-------------+
|emotion|       words_stemmed|         rawFeatures|emotion_index|
+-------+--------------------+--------------------+-------------+
|    joy|[oh, seem, weeken...|(262144,[18184,13...|          1.0|
|    joy|[joy, around, us,...|(262144,[64859,10...|          1.0|
|    joy|[knacker, prep, t...|(262144,[29129,11...|          1.0|
|    joy|[believ, day, lef...|(262144,[121517,1...|          1.0|
|    joy|  [footbal, banquet]|(262144,[94429,13...|          1.0|
|    joy|[final, good, mor...|(262144,[61899,11...|          1.0|
|    joy|[know, dissert, t...|(262144,[16673,13...|          1.0|
|    joy|[ever, want, writ...|(262144,[89554,14...|          1.0|
|    joy|[first, time, yea...|(262144,[34194,12...|          1.0|
|    joy|[let, christma, c...|(262144,[129434,1...|          1.0|
|    joy|[omg, feel, like,...|(262144,[17290,61...|          1.0|
|    joy|       [final, exam]|(262144,[66864,24...|          1.0|
|    joy|[

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictions = model.transform(df_test_data_features_part_A_indexed)
predictions.select("emotion", "emotion_index", 
    "probability", "prediction").show()
evaluator =\
    MulticlassClassificationEvaluator(labelCol="emotion_index",\
    predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+-------+-------------+--------------------+----------+
|emotion|emotion_index|         probability|prediction|
+-------+-------------+--------------------+----------+
|    joy|          3.0|[1.56816316856732...|       1.0|
|    joy|          3.0|[6.37152517618134...|       1.0|
|    joy|          3.0|[0.01054223939004...|       1.0|
|    joy|          3.0|[1.38691470194256...|       1.0|
|    joy|          3.0|[6.02388942501134...|       1.0|
|    joy|          3.0|[1.22817366994222...|       1.0|
|    joy|          3.0|[1.11414190375292...|       1.0|
|    joy|          3.0|[7.21375843844416...|       1.0|
|    joy|          3.0|[7.70266821494761...|       1.0|
|    joy|          3.0|[1.41328011102548...|       1.0|
|    joy|          3.0|[1.47574756610789...|       1.0|
|    joy|          3.0|[2.53695963824126...|       1.0|
|    joy|          3.0|[1.34757267566723...|       1.0|
|    joy|          3.0|[6.00213290733136...|       1.0|
|    joy|          3.0|[5.52010778647383...|    